In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Define the directory path to save graphs
output_dir = r"C:\Users\Aditya\Desktop\FyPro\FYP\Financial-Management-Dataset-\Code\Kishan\Code\graph"
os.makedirs(output_dir, exist_ok=True)


# Step 2: TCS candy profits list
data = {
    "Tata Consultancy Services Ltd": {
        'Quarter': ['Dec 13', 'Mar 14', 'Jun 14', 'Sep 14', 'Dec 14', 'Mar 15', 'Jun 15', 'Sep 15', 
                    'Dec 15', 'Mar 16', 'Jun 16', 'Sep 16', 'Dec 16', 'Mar 17', 'Jun 17', 'Sep 17', 
                    'Dec 17', 'Mar 18', 'Jun 18', 'Sep 18', 'Dec 18', 'Mar 19', 'Jun 19', 'Sep 19', 
                    'Dec 19', 'Mar 20', 'Jun 20', 'Sep 20', 'Dec 20', 'Mar 21', 'Jun 21', 'Sep 21', 
                    'Dec 21', 'Mar 22', 'Jun 22', 'Sep 22', 'Dec 22', 'Mar 23', 'Jun 23', 'Sep 23', 
                    'Dec 23', 'Mar 24', 'Jun 24', 'Sep 24', 'Dec 24'],
        'Net Profit': [4776.76, 4628.52, 5096.87, 4991.68, 5711.15, 3457.26, 5633.0, 5723.0, 
                       5991.0, 5728.0, 5778.0, 5958.0, 6084.0, 5833.0, 6670.0, 5746.0, 
                       6069.0, 6756.0, 7136.0, 7620.0, 7486.0, 7823.0, 8714.0, 8636.0, 
                       7683.0, 8227.0, 6096.0, 7448.0, 9242.0, 8174.0, 8281.0, 10152.0, 
                       9959.0, 9795.0, 8588.0, 10059.0, 10659.0, 9800.0, 10484.0, 10929.0, 
                       10753.0, 11393.0, 12115.0, 12994.0, 11832.0]
    },
    "Wipro Ltd": {
        'Quarter': ['Dec 13', 'Mar 14', 'Jun 14', 'Sep 14', 'Dec 14', 'Mar 15', 'Jun 15', 'Sep 15', 
                    'Dec 15', 'Mar 16', 'Jun 16', 'Sep 16', 'Dec 16', 'Mar 17', 'Jun 17', 'Sep 17', 
                    'Dec 17', 'Mar 18', 'Jun 18', 'Sep 18', 'Dec 18', 'Mar 19', 'Jun 19', 'Sep 19', 
                    'Dec 19', 'Mar 20', 'Jun 20', 'Sep 20', 'Dec 20', 'Mar 21', 'Jun 21', 'Sep 21', 
                    'Dec 21', 'Mar 22', 'Jun 22', 'Sep 22', 'Dec 22', 'Mar 23', 'Jun 23', 'Sep 23', 
                    'Dec 23', 'Mar 24', 'Jun 24', 'Sep 24', 'Dec 24'],
        'Net Profit': [2047.9, 2353.1, 2067.2, 1992.0, 1992.3, 2141.6, 2063.0, 2153.1, 
                       2011.1, 1973.3, 2007.7, 1932.1, 1918.5, 2303.5, 2026.8, 2134.3, 
                       1804.5, 1757.2, 1922.2, 1607.4, 2518.4, 1566.0, 2014.1, 2247.0, 
                       2261.3, 2158.3, 2192.6, 2244.0, 2611.3, 3013.0, 1804.1, 2387.4, 
                       2465.0, 5478.8, 2216.6, 2282.9, 2413.1, 2264.1, 2587.8, 2061.3, 
                       2022.9, 2446.6, 2374.6, 2713.5, 2812.1]
    },
    "3i Infotech Ltd": {
        'Quarter': ['Dec 13', 'Mar 14', 'Jun 14', 'Sep 14', 'Dec 14', 'Mar 15', 'Jun 15', 'Sep 15', 
                    'Dec 15', 'Mar 16', 'Jun 16', 'Sep 16', 'Dec 16', 'Mar 17', 'Jun 17', 'Sep 17', 
                    'Dec 17', 'Mar 18', 'Jun 18', 'Sep 18', 'Dec 18', 'Mar 19', 'Jun 19', 'Sep 19', 
                    'Dec 19', 'Mar 20', 'Jun 20', 'Sep 20', 'Dec 20', 'Mar 21', 'Jun 21', 'Sep 21', 
                    'Dec 21', 'Mar 22', 'Jun 22', 'Sep 22', 'Dec 22', 'Mar 23', 'Jun 23', 'Sep 23', 
                    'Dec 23', 'Mar 24', 'Jun 24', 'Sep 24', 'Dec 24'],
        'Net Profit': [-97.48, -139.44, -121.33, -105.21, -100.4, -451.18, -400.96, -101.52, 
                       -111.91, -17.65, -832.58, 15.51, 14.25, 56.79, 5.72, 17.7, 
                       -130.09, -2.1, 27.75, 39.54, 78.61, 110.25, 22.5, -2.03, 
                       28.16, 30.0, 71.76, 24.04, 2.58, 19.89, 277.91, 21.75, 
                       -12.42, -21.77, 20.86, 25.54, 6.77, 41.5, -21.56, -15.3, 
                       -114.78, -642.91, -45.64, -9.0, 13.53]
    }
}


# Convert quarterly data to DataFrame
df_dict = {}
for company, values in data.items():
    temp_df = pd.DataFrame(values)
    temp_df['Quarter'] = pd.to_datetime(temp_df['Quarter'], format='%b %y')
    temp_df.set_index('Quarter', inplace=True)
    df_dict[company] = temp_df

# Define SARIMA order
order = (2, 0, 0)
seasonal_order = (0, 0, 1, 4)

# Process each company
for company, df in df_dict.items():
    print(f"Processing {company}...")
    
    # Split data into training and testing
    train_data = df.loc[:'2022-12-31']
    test_data = df.loc['2023-03-31':]
    
    # Train SARIMA model
    model = SARIMAX(train_data['Net Profit'], order=order, seasonal_order=seasonal_order,
                    enforce_stationarity=False, enforce_invertibility=False)
    results = model.fit()
    
    # Forecast the test period
    test_forecast = results.get_forecast(steps=len(test_data))
    test_predicted = test_forecast.predicted_mean
    test_conf_int = test_forecast.conf_int()
    
    # Compute error metrics
    mae = mean_absolute_error(test_data['Net Profit'], test_predicted)
    rmse = np.sqrt(mean_squared_error(test_data['Net Profit'], test_predicted))
    print(f"MAE: {mae:.2f} Rs. Cr., RMSE: {rmse:.2f} Rs. Cr.\n")
    
    # Forecast for 2025
    future_forecast = results.get_forecast(steps=len(test_data) + 4)
    future_predicted = future_forecast.predicted_mean[-4:]
    future_dates = pd.date_range(start='2025-03-31', periods=4, freq='Q')
    
    # Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(train_data.index, train_data['Net Profit'], label='Training Profits', color='blue')
    plt.plot(test_data.index, test_data['Net Profit'], label='Real Test Profits', color='green')
    plt.plot(test_data.index, test_predicted, label='Guessed Test Profits', color='orange', linestyle='--')
    plt.plot(future_dates, future_predicted, label='Future Guesses (2025)', color='red', linestyle='--')
    plt.fill_between(test_data.index, test_conf_int.iloc[:, 0], test_conf_int.iloc[:, 1], color='yellow', alpha=0.2)
    plt.title(f'{company} Net Profit: Real vs Guessed')
    plt.xlabel('Time')
    plt.ylabel('Profit (Rs. Cr.)')
    plt.legend()
    plt.grid(True)
    
    # Save plot
    save_path = os.path.join(output_dir, f"{company.replace(' ', '_')}_forecast.png")
    plt.savefig(save_path)
    plt.close()
    
    # Print future predictions
    print(f"Guessed Profits for 2025 ({company}):")
    for date, value in zip(future_dates, future_predicted):
        print(f"{date.strftime('%b %y')}: {value:.2f} Rs. Cr.")
    print("\nGraph saved at:", save_path)


Processing Tata Consultancy Services Ltd...
MAE: 369.80 Rs. Cr., RMSE: 536.48 Rs. Cr.



c:\Users\Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-DEC will be used.
  self._init_dates(dates, freq)
c:\Users\Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-DEC will be used.
  self._init_dates(dates, freq)
C:\Users\Aditya\AppData\Local\Temp\ipykernel_11512\2626325941.py:98: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  future_dates = pd.date_range(start='2025-03-31', periods=4, freq='Q')


Guessed Profits for 2025 (Tata Consultancy Services Ltd):
Mar 25: 12215.03 Rs. Cr.
Jun 25: 12441.72 Rs. Cr.
Sep 25: 12672.77 Rs. Cr.
Dec 25: 12908.05 Rs. Cr.

Graph saved at: C:\Users\Aditya\Desktop\FyPro\FYP\Financial-Management-Dataset-\Code\Kishan\Code\graph\Tata_Consultancy_Services_Ltd_forecast.png
Processing Wipro Ltd...
MAE: 729.17 Rs. Cr., RMSE: 1085.23 Rs. Cr.



c:\Users\Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-DEC will be used.
  self._init_dates(dates, freq)
c:\Users\Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-DEC will be used.
  self._init_dates(dates, freq)
C:\Users\Aditya\AppData\Local\Temp\ipykernel_11512\2626325941.py:98: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  future_dates = pd.date_range(start='2025-03-31', periods=4, freq='Q')


Guessed Profits for 2025 (Wipro Ltd):
Mar 25: 2458.36 Rs. Cr.
Jun 25: 2473.99 Rs. Cr.
Sep 25: 2481.95 Rs. Cr.
Dec 25: 2493.27 Rs. Cr.

Graph saved at: C:\Users\Aditya\Desktop\FyPro\FYP\Financial-Management-Dataset-\Code\Kishan\Code\graph\Wipro_Ltd_forecast.png
Processing 3i Infotech Ltd...
MAE: 122.53 Rs. Cr., RMSE: 249.47 Rs. Cr.



c:\Users\Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-DEC will be used.
  self._init_dates(dates, freq)
c:\Users\Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-DEC will be used.
  self._init_dates(dates, freq)
C:\Users\Aditya\AppData\Local\Temp\ipykernel_11512\2626325941.py:98: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  future_dates = pd.date_range(start='2025-03-31', periods=4, freq='Q')


Guessed Profits for 2025 (3i Infotech Ltd):
Mar 25: 0.14 Rs. Cr.
Jun 25: 0.06 Rs. Cr.
Sep 25: 0.03 Rs. Cr.
Dec 25: 0.01 Rs. Cr.

Graph saved at: C:\Users\Aditya\Desktop\FyPro\FYP\Financial-Management-Dataset-\Code\Kishan\Code\graph\3i_Infotech_Ltd_forecast.png
